# LSTM weeek

Torch LSTM tutorial: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

In [72]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd
import numpy as np
import re
import json
import ast
torch.manual_seed(1)

# Подготовим данные для обучения NER

In [5]:
df = pd.read_csv('datasets/for_train.csv')
df.head()

,text,new_text,task_type,person,time
0,Создай задачу : Подготовить отчет к 15 : 00 за...,создавать задача : подготовить отчет к 15 : 00...,['создавать задача : подготовить отчет к 15 : ...,['создавать задача : подготовить отчет к 15 : ...,['создавать задача : подготовить отчет к 15 : ...
1,Создай задание : Отправить приглашения на собр...,создавать задание : отправлять приглашение на ...,['создавать задание : отправлять приглашение н...,['создавать задание : отправлять приглашение н...,['создавать задание : отправлять приглашение н...
2,Создай задачу : Разработать презентацию к 18 :...,создавать задача : разрабатывать презентация к...,['создавать задача : разрабатывать презентация...,['создавать задача : разрабатывать презентация...,['создавать задача : разрабатывать презентация...
3,Назначь обязанность : Провести собрание с кома...,назначать обязанность : проводить собрание с к...,['назначать обязанность : проводить собрание с...,['назначать обязанность : проводить собрание с...,['назначать обязанность : проводить собрание с...
4,Поставь задачу : Подготовить план мероприятий ...,поставлять задача : подготовить план мероприят...,['поставлять задача : подготовить план меропри...,['поставлять задача : подготовить план меропри...,['поставлять задача : подготовить план меропри...


In [96]:
def get_entities(row):
  entities = list()
  if isinstance(row, str):
    m = re.search("{'entities.*}", row)
    ent_dict = ast.literal_eval(m.group(0))
    for x in ent_dict['entities']:
      entities.extend(row[x[0]+2:x[1]+2].split())
  return entities

data = list()
for i, row in df.iterrows():
  try:
    sent = row['new_text'].split()
    tokens = ['O'] * len(sent)
    persons = get_entities(row['person'])
    for person in persons:
      tokens[sent.index(person)] = 'PERSON'
    times = get_entities(row['time'])
    for time in times:
      tokens[sent.index(time)] = 'TIME'
    task_types = get_entities(row['task_type'])
    for task_type in task_types:
      tokens[sent.index(task_type)] = 'TASK'
    data.append((sent, tokens))
  except:
    print(type(row['person']))
    print(row['person'])
    raise

In [97]:
print(data[0])
print(data[20])
print(data[100])

(['создавать', 'задача', ':', 'подготовить', 'отчет', 'к', '15', ':', '00', 'завтра', 'на', 'анна'], ['TASK', 'TASK', 'O', 'O', 'O', 'O', 'TIME', 'O', 'TIME', 'TIME', 'O', 'PERSON'])
(['создавать', 'задача', ':', 'подготовить', 'аналитический', 'отчет', 'к', '9', ':', '00', 'утро', 'на', 'катя'], ['TASK', 'TASK', 'O', 'O', 'O', 'O', 'O', 'TIME', 'O', 'TIME', 'TIME', 'O', 'O'])
(['создавать', 'задание', ':', 'обновлять', 'документация', 'к', '12', ':', '00', 'утро', 'на', 'наталья'], ['TASK', 'TASK', 'O', 'O', 'O', 'O', 'TIME', 'O', 'TIME', 'TIME', 'O', 'PERSON'])


# LSTM for NER

In [98]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

split_idx = int(len(data)*0.9)
training_data = data[:split_idx]
test_data = data[split_idx:]
word_to_ix = {}

In [131]:
for sent, tags in data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)

print(word_to_ix)

tag_to_ix = {"O": 0, "TASK": 1, "PERSON": 2, "TIME": 3}

print(tag_to_ix)

EMBEDDING_DIM = 6
HIDDEN_DIM = 6

{'создавать': 0, 'задача': 1, ':': 2, 'подготовить': 3, 'отчет': 4, 'к': 5, '15': 6, '00': 7, 'завтра': 8, 'на': 9, 'анна': 10, 'задание': 11, 'отправлять': 12, 'приглашение': 13, 'собрание': 14, '10': 15, 'утро': 16, 'иван': 17, 'разрабатывать': 18, 'презентация': 19, '18': 20, 'сегодня': 21, 'сергей': 22, 'назначать': 23, 'обязанность': 24, 'проводить': 25, 'с': 26, 'команда': 27, 'в': 28, '14': 29, '30': 30, 'ольга': 31, 'поставлять': 32, 'план': 33, 'мероприятие': 34, '12': 35, 'следующий': 36, 'день': 37, 'наталья': 38, 'задавать': 39, 'письмо': 40, 'уточнение': 41, '16': 42, 'василий': 43, 'проверять': 44, 'документация': 45, '9': 46, 'денис': 47, 'давать': 48, 'поручение': 49, 'бюджет': 50, 'проект': 51, '17': 52, 'екатерина': 53, 'тренинг': 54, 'сотрудник': 55, '11': 56, 'артур': 57, 'резюме': 58, 'марина': 59, 'до': 60, 'конец': 61, 'стратегия': 62, 'маркетинг': 63, 'согласовывать': 64, 'график': 65, 'работа': 66, 'о': 67, 'выполнение': 68, 'обучение': 69, 'новый': 70, 'корпор

In [132]:
# Model:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [133]:
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    print(word_to_ix)

tensor([ 0,  1,  2,  3,  4,  5,  6,  2,  7,  8,  9, 10])
{'создавать': 0, 'задача': 1, ':': 2, 'подготовить': 3, 'отчет': 4, 'к': 5, '15': 6, '00': 7, 'завтра': 8, 'на': 9, 'анна': 10, 'задание': 11, 'отправлять': 12, 'приглашение': 13, 'собрание': 14, '10': 15, 'утро': 16, 'иван': 17, 'разрабатывать': 18, 'презентация': 19, '18': 20, 'сегодня': 21, 'сергей': 22, 'назначать': 23, 'обязанность': 24, 'проводить': 25, 'с': 26, 'команда': 27, 'в': 28, '14': 29, '30': 30, 'ольга': 31, 'поставлять': 32, 'план': 33, 'мероприятие': 34, '12': 35, 'следующий': 36, 'день': 37, 'наталья': 38, 'задавать': 39, 'письмо': 40, 'уточнение': 41, '16': 42, 'василий': 43, 'проверять': 44, 'документация': 45, '9': 46, 'денис': 47, 'давать': 48, 'поручение': 49, 'бюджет': 50, 'проект': 51, '17': 52, 'екатерина': 53, 'тренинг': 54, 'сотрудник': 55, '11': 56, 'артур': 57, 'резюме': 58, 'марина': 59, 'до': 60, 'конец': 61, 'стратегия': 62, 'маркетинг': 63, 'согласовывать': 64, 'график': 65, 'работа': 66, 'о': 6

In [134]:
targets = prepare_sequence(tags, tag_to_ix)
print(targets)

tensor([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3])


In [135]:
# Train the model
from sklearn.metrics import accuracy_score

model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

train_losses = list()
test_losses = list()
acc_scores = list()

for epoch in range(1, 101, 1):
    print(epoch)
    loss_list=list()
    for sentence, tags in training_data:
        model.zero_grad()
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)
        tag_scores = model(sentence_in)
        loss = loss_function(tag_scores, targets)
        loss_list.append(loss.item())
        loss.backward()
        optimizer.step()
    with torch.no_grad():
        Y_shuffled, Y_preds, losses = [],[],[]
        for sentence, tags in test_data:
            sentence_in = prepare_sequence(sentence, word_to_ix)
            targets = prepare_sequence(tags, tag_to_ix)
            preds = model(sentence_in)
            loss = loss_function(preds, targets)
            losses.append(loss.item())
            Y_shuffled.append(targets)
            Y_preds.append(preds.argmax(dim=-1))

        Y_shuffled = torch.cat(Y_shuffled)
        Y_preds = torch.cat(Y_preds)
        test_loss = torch.tensor(losses).mean()
        acc_score = accuracy_score(Y_shuffled.detach().numpy(), Y_preds.detach().numpy())
        train_loss = np.mean(loss_list)
        train_losses.append(train_loss)
        test_losses.append(test_loss)
        acc_scores.append(acc_score)
        print("Train Loss : {:.3f}".format(train_loss))
        print("Valid Loss : {:.3f}".format(test_loss))
        print("Valid Acc  : {:.3f}".format(acc_score))


1
Train Loss : 0.758
Valid Loss : 0.542
Valid Acc  : 0.834
2
Train Loss : 0.475
Valid Loss : 0.460
Valid Acc  : 0.834
3
Train Loss : 0.384
Valid Loss : 0.425
Valid Acc  : 0.846
4
Train Loss : 0.333
Valid Loss : 0.417
Valid Acc  : 0.853
5
Train Loss : 0.293
Valid Loss : 0.418
Valid Acc  : 0.860
6
Train Loss : 0.260
Valid Loss : 0.419
Valid Acc  : 0.859
7
Train Loss : 0.232
Valid Loss : 0.417
Valid Acc  : 0.860
8
Train Loss : 0.209
Valid Loss : 0.412
Valid Acc  : 0.860
9
Train Loss : 0.188
Valid Loss : 0.407
Valid Acc  : 0.869
10
Train Loss : 0.171
Valid Loss : 0.404
Valid Acc  : 0.874
11
Train Loss : 0.157
Valid Loss : 0.399
Valid Acc  : 0.873
12
Train Loss : 0.146
Valid Loss : 0.392
Valid Acc  : 0.874
13
Train Loss : 0.135
Valid Loss : 0.384
Valid Acc  : 0.870
14
Train Loss : 0.125
Valid Loss : 0.380
Valid Acc  : 0.869
15
Train Loss : 0.117
Valid Loss : 0.380
Valid Acc  : 0.873
16
Train Loss : 0.110
Valid Loss : 0.382
Valid Acc  : 0.877
17
Train Loss : 0.103
Valid Loss : 0.385
Valid Ac

In [137]:
for test_case in test_data:
  test_case = test_case[0]
  print(' '.join(test_case))

  with torch.no_grad():
      inputs = prepare_sequence(test_case, word_to_ix)
      tag_scores = model(inputs)
      results = [(tag_scores == max(i)).nonzero(as_tuple=False)[0][1].item() for i in tag_scores]
      ix_to_tag = {v:k for k,v in tag_to_ix.items()}
      translations = [ix_to_tag[i] for i in results]

  print("TASK: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="TASK"])))
  print("PERSON: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="PERSON"])))
  print("TIME: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="TIME"])))
  print("DESC:{}".format(' '.join(test_case)))
  print('')

создавать задача : олеся должный предоставлять эскиз сувенирный продукция завтра к 11 : 00
TASK: создавать задача
PERSON: олеся
TIME: 11 00
DESC:создавать задача : олеся должный предоставлять эскиз сувенирный продукция завтра к 11 : 00

создавать задача : попросить дима разрабатывать несколько вариант баннер к 1 май к 10 : 00 утро
TASK: создавать задача
PERSON: 
TIME: 1 май 10 00 утро
DESC:создавать задача : попросить дима разрабатывать несколько вариант баннер к 1 май к 10 : 00 утро

создавать задача : напоминать виктор о встреча по ребрендинг компания послезавтра в 15 : 30
TASK: создавать задача напоминать
PERSON: виктор
TIME: послезавтра 15 30
DESC:создавать задача : напоминать виктор о встреча по ребрендинг компания послезавтра в 15 : 30

создавать задача : миша нужно подготовить moodboard для фотосессия домашний одежда к понедельник к 14 : 00
TASK: создавать задача
PERSON: 
TIME: 14 00
DESC:создавать задача : миша нужно подготовить moodboard для фотосессия домашний одежда к понеде

In [144]:
for test_case in ['создавать задача : олеся должный предоставлять эскиз новый карнавальный костюм завтра к 11 : 00'.split()]:
  print(' '.join(test_case))

  with torch.no_grad():
      inputs = prepare_sequence(test_case, word_to_ix)
      tag_scores = model(inputs)
      results = [(tag_scores == max(i)).nonzero(as_tuple=False)[0][1].item() for i in tag_scores]
      ix_to_tag = {v:k for k,v in tag_to_ix.items()}
      translations = [ix_to_tag[i] for i in results]

  print("TASK: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="TASK"])))
  print("PERSON: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="PERSON"])))
  print("TIME: {}".format(' '.join([test_case[i] for i in range(len(test_case)) if translations[i]=="TIME"])))
  print("DESC:{}".format(' '.join(test_case)))
  print('')

создавать задача : олеся должный предоставлять эскиз новый карнавальный костюм завтра к 11 : 00


KeyError: ignored

In [145]:
torch.save(model, "LSTM_NER.pth")